[website](https://boardgamegeek.com/boardgame/224517/brass-birmingham/stats)

In [3]:
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    WebDriverException,
    InvalidSessionIdException
)
import time
from tqdm import tqdm
import logging
import json
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

allboardgames = 0
all_games_links = []
boardgames = []

page_one = 1 
page_end = 2

try:
    for page in range(page_one, page_end):
        url = f"https://boardgamegeek.com/browse/boardgame/page/{page}"
        driver.get(url)
        time.sleep(3)

        try:
            game_links_per_page = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@class="primary"]')]
            all_games_links.extend(game_links_per_page)
        except (NoSuchElementException, StaleElementReferenceException) as e:
                print(f"Error on page {page}: {e}")
except (TimeoutException, WebDriverException, InvalidSessionIdException) as e:
    print(f"Navigation error: {e}")

game_links = 'GoToGames.txt'
with open(game_links, 'w') as file:
    for game_link in all_games_links:
         file.write(game_link + '\n')

print(f"The links to all {len(all_games_links)} board games has been collected and stored successfully")

with tqdm(total=len(all_games_links), desc="Games Scraped") as pbar:

    try:
        for href in all_games_links:
            driver.get(href)
            time.sleep(5)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

        #Section Zero
        #xt/players
            players = driver.find_elements(By.XPATH, '//span[@ng-if="::geekitemctrl.geekitem.data.item.minplayers > 0 || geekitemctrl.geekitem.data.item.maxplayers > 0"]')

            if players:
                players_text = players[0].text.strip()
                
                if '–' in players_text:
                    min_players = int(players_text.split('–')[0])
                    max_players = int(players_text.split('–')[1])
                else:
                    min_players = max_players = int(players_text)
            else:
                min_players = max_players = None

        #xt/timing
            timing = driver.find_elements(By.XPATH, '//span[@min="::geekitemctrl.geekitem.data.item.minplaytime" and @max="::geekitemctrl.geekitem.data.item.maxplaytime"]')

            if timing:
                timing_text = timing[0].text.strip()
                
                if '–' in timing_text:
                    min_time = int(timing_text.split('–')[0])
                    max_time = int(timing_text.split('–')[1])
                else:
                    min_time = max_time = int(timing_text)
            else:
                min_time = max_time = None


        #Section One
        #click 
            SeeFullCredits = WebDriverWait(driver,10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@ui-sref="geekitem.credits"]'))
            )
            SeeFullCredits.click()

        #wait for game_name and release year element
            WebDriverWait(driver,10).until(
            EC.visibility_of_element_located((By.XPATH, '//span[@ng-bind-html="creditsctrl.geekitem.data.item[info.keyname]|to_trusted"]'))
            )

        #xt/game_name, release year
            spans = driver.find_elements(By.XPATH, '//span[@ng-bind-html="creditsctrl.geekitem.data.item[info.keyname]|to_trusted"]')
            game_name = spans[0].text.strip()
            release_year = int(spans[1].text.strip())
                
            credits = driver.find_elements(By.XPATH, '//div[@ng-if="info.datatype == \'geekitem_linkdata\'"]')
            
                    # designers_root = credits[0].text
                    # if '\n' in designers_root:
                    #     designers = designers_root.replace('\n', ', ').split(", ")
                    # else:
                    #     designers = [designers_root.strip()]


                artists_root = credits[2].text
                if '\n' in artists_root:
                    artists = artists_root.replace('\n', ', ').split(", ")
                else:
                    artists = [artists_root.strip()]

                publishers_root = credits[3].text
                if '\n' in publishers_root:
                    publishers = publishers_root.replace('\n', ', ').split(", ")
                else:
                    publishers = [publishers_root.strip()]

                developer_root = credits[4].text
                if '\n' in developer_root:
                    developer = developer_root.replace('\n', ', ').split(", ")
                else:
                    developer = developer_root.strip()

                graphic_designer_root = credits[5].text
                if '\n' in graphic_designer_root:
                    graphic_designer = graphic_designer_root.replace('\n', ', ').split(", ")
                else:
                    graphic_designer = graphic_designer_root.strip()

                sculptor_root = credits[6].text
                if '\n' in sculptor_root:
                    sculptor = sculptor_root.replace('\n', ', ').split(", ")
                else:
                    sculptor = sculptor_root.strip()

                editor_root = credits[7].text
                if '\n' in editor_root:
                    editor = editor_root.replace('\n', ', ').split(", ")
                else:
                    editor = editor_root.strip()

                writer_root = credits[8].text
                if '\n' in writer_root:
                    writer = writer_root.replace('\n', ', ').split(", ")
                else:
                    writer = writer_root.strip()

                insert_designer_root = credits[9].text
                if '\n' in insert_designer_root:
                    insert_designer = insert_designer_root.replace('\n', ', ').split(", ")
                else:
                    insert_designer = insert_designer_root.strip()

                categories_root = credits[10].text
                if '\n' in categories_root:
                    categories = categories_root.replace('\n', ', ').split(", ")
                else:
                    categories = categories_root.strip()

                mechanisms_root = credits[11].text
                if '\n' in mechanisms_root:
                    mechanisms = mechanisms_root.replace('\n', ', ').split(", ")
                else:
                    mechanisms = mechanisms_root.strip()

                family_root = credits[12].text
                if '\n' in family_root:
                    family = family_root.replace('\n', ', ').split(", ")
                else:
                    family = family_root.strip()

                per_boardgame = {
                    "boardgame":game_name,
                    "release_year":release_year,
                    "min_players":minimum_players,
                    "max_players":maximum_players,
                    "min_playing_time":minimum_playing_time,
                    "max_playing_time":maximum_playing_time,
                    "designers":designers,
                    "solo_designer":solo_designer,
                    "artists":artists,
                    "publishers":publishers,
                    "developers":developer,
                    "graphic_designers":graphic_designer,
                    "sculptor":sculptor,
                    "editor":editor,
                    "writer":writer,
                    "insert_designer":insert_designer,
                    "categories":categories,
                    "mechanisms":mechanisms,
                    "family":family
                }

                boardgames.append(per_boardgame)
                allboardgames+=1
                pbar.update(1)


            # SeeGameStats = WebDriverWait(driver,5).until(
            #     EC.element_to_be_clickable((By.XPATH, '//a[@ui-sref="geekitem.stats({})"]'))
            # )
            # SeeGameStats.click()

            



            finally:
                driver.quit()

with open("boardgamegeek.json", "w", encoding="utf-8") as f:
    json.dump(boardgames, f, indent=4, ensure_ascii=False)

In [ ]:
#Game-Stats XTract test

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://boardgamegeek.com/boardgame/224517/brass-birmingham/stats"
driver.get(url)

stats_elem = driver.find_elements(By.XPATH, '//ul[@class="outline fs-responsive-sm outline-border-col-xs"]')

try:
    game_stats = stats_elem[0].text.split('\n')
    average_rating = float(game_stats[1])
    num_of_ratings = int(game_stats[3].replace(',',''))
    std_deviation = float(game_stats[5])
    weight_over_five = float(game_stats[7].split(' / ')[0])
    comments = int(game_stats[9].replace(',',''))
    fans = int(game_stats[11].replace(',',''))
    page_views = int(game_stats[13].replace(',',''))

    game_ranks = stats_elem[1].text.splitlines()
    overall_rank = int(game_ranks[1].replace(',',''))
    
    play_stats = stats_elem[2].text.splitlines()
    all_time_plays = play_stats[1]

    collection_stats = stats_elem[3].text.split('\n')
    own = int(collection_stats[1].replace(',',''))
    previously_owned = int(collection_stats[3].replace(',',''))
    for_trade = int(collection_stats[5].replace(',',''))
    want_in_trade = int(collection_stats[7].replace(',',''))
    wishlist = int(collection_stats[9].replace(',',''))
except Exception as e:
    print("Error Scraping Stats")


driver.quit()
# finally:
#     driver.quit()
#     duration = time.time() - start_time
#     logging.info(f"Scraper finished in {duration:.2f} seconds.")

In [11]:
print( minimum_players,
      maximum_players,
      minimum_playing_time,
      maximum_playing_time,
      game_name,
      release_year,
      designers,
      solo_designer,
      artists,
      publishers,
      developer,
      graphic_designer,
      sculptor,
      editor,
      writer,
      insert_designer,
      categories,
      mechanisms,
      family
)

2 4 60 120 Brass: Birmingham 2018 ['Gavan Brown', 'Matt Tolman', 'Martin Wallace'] N/A ['Gavan Brown', 'Lina Cossette', 'David Forest', 'Gui Landgraf', 'Damien Mammoliti', 'Matt Tolman'] ['Roxley', 'Arclight Games', 'Board Game Rookie', 'BoardM Factory', 'CMON Global Limited', 'Conclave Editora', 'CrowD Games', 'Dexker Games', 'Funforge', 'Gém Klub Kft.', 'Ghenos Games', 'Giant Roc', 'Lanlalen', 'Lord of Boards', 'Maldito Games', 'PHALANX', 'Rebel Sp. z o.o.', 'TLAMA games', 'White Goblin Games', '盒拍工作室 Hepa Studio'] N/A N/A N/A N/A N/A N/A ['Age of Reason', 'Economic', 'Industry / Manufacturing', 'Post-Napoleonic', 'Trains', 'Transportation'] ['Chaining', 'End Game Bonuses', 'Hand Management', 'Income', 'Loans', 'Market', 'Multi-Use Cards', 'Network and Route Building', 'Ownership', 'Tags', 'Tech Trees / Tech Tracks', 'Tile Placement', 'Turn Order: Stat-Based', 'Variable Set-up'] ['Cities: Birmingham (England)', 'Components: Map (Regional scale)', 'Components: Multi-Use Cards', 'Count